<center><img widh="70%" src="img/logo.png"></center>

# **Observatório Social - Prefeitura Municipal de Indaial/SC**
### <font color='#232679'> *Webscrapping com Python | Criação e alimentação de DW*
*by [GUILHERME MADEIRA BAECE](https://github.com/GMBaece)

In [1]:
# Bibliotecas utilizadas
import pandas as pd
import bs4
import requests
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError
import time
import pyodbc

In [2]:
# Site da prefeitura de Indaial
url = 'https://www.camaraindaial.sc.gov.br/pg/proposicoes'

In [3]:
# User Agent
agente = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'

# dicionário criado para armazenar o valor User Agent
headers = {'User-Agent': agente}

In [4]:
# Função para capturar dados do site e tratar erros
def consultaWeb(url):
    try:
        req = Request(url, headers = headers)
        response = urlopen(req)
        return response.read()
    
    except:
        pass

In [5]:
# Função para transofrmar os dados do site em algo mais leggpivel, utilizando a biblioteca bs4
def captura_html_pagina(url):
    html = consultaWeb(url)
    soup = bs4.BeautifulSoup(html, 'html.parser')
    return soup

In [6]:
url = 'https://www.legislador.com.br//LegisladorWEB.ASP?WCI=ProposicaoTexto&ID=3&TPProposicao=1&nrProposicao=853&aaProposicao=2022'

In [7]:
# Função para criar um dicionário com os dados encontrados no site por meio da extração do conteúdo alocado nos elementos identificados no código HTML
def DictDados(html):
    dt = html.findAll('dt')
    dd = html.findAll('dd')
    dic = {}
    for i in range(len(dt)):
        x = dt[i].get_text()
        y = dd[i].get_text()
        dic[x] = y
    return dic

In [8]:
# Função para inserir os dados restantes no dicionário criado acima, inserindo o número da proposição, o ano e buscando também o texto que está alocado em um outro elemento no HTML
def Conteudo(proposicao,ano):
    
    url = 'https://www.legislador.com.br//LegisladorWEB.ASP?WCI=ProposicaoTexto&ID=3&TPProposicao=1&nrProposicao='+str(proposicao)+'&aaProposicao='+str(ano)
    html = captura_html_pagina(url)
    
    try:
        dt = html.findAll('dt')
        if len(dt) == 0:                            #condicional para evitar que respostas vazias sejam retornadas 
            new_dict = DictDados(html)
            return new_dict
        else:
                        
            new_dict = DictDados(html)
            new_dict['Proposição'] = proposicao
            new_dict['Ano'] = ano
            new_dict['Texto'] = html.p.get_text()
            
            return new_dict
    except:
        pass


In [9]:
# Teste da função 
# Conteudo(853,2022)
html = captura_html_pagina(url)
DictDados(html)

{'Reunião': '03/11/2022',
 'Situação': 'Entrada no Expediente',
 'Assunto': 'Limpeza, Macadamização, Patrolamento, Retificação; Alargamento',
 'Autor': 'Vereador Anderson Luz dos Santos.'}

In [ ]:
#Função que reúne os processoas anteriores e retorna em uma tabela organizada com o método DataFrame do Pandas
def TabelaResultados(inicio, quantidade, ano, erros_admissiveis, segundos_espera):
    
    ultima_consulta = inicio + quantidade -1
    
    #erros
    erros = 0
    
    #variaveis para loop
    i = 1
    lista = []
    
    while inicio <= ultima_consulta and erros <= erros_admissiveis:
        
        try:
            x = Conteudo(inicio, ano)
            lista = lista + [Conteudo(inicio, ano)]
        except:
            erros += 1
            pass
    
        time.sleep(segundos_espera)
        
        # carregamento incremental das variáveis
        inicio += 1
        i += 1
        
    return pd.DataFrame(lista)

In [ ]:
# chamar a funçãoTabelaResultados
TabelaResultados(12,10,2022,2,0.5)

In [10]:
#Função para utilizar a linguagem SQL utilizando a biblioteca pyodbc
def SQLSelect(query):
    
    conn = pyodbc.connect('Trusted_Connection=yes',
                      driver = '{ODBC Driver 17 for SQL Server}',
                      server = 'localhost',
                      database = 'Indaial')
    
    out = pd.read_sql_query(query, conn)
    return out

In [11]:
# Função para inserir dados em um banco de dados SQL Server com criação de cursor para inserir linha por linha os dados
def SQLInsertProposicoes(TabelaProposicoes):
    
    base = pd.DataFrame(columns = ['Reunião','Deliberação', 'Situação', 'Assunto', 'Autor', 'Proposição', 'Ano', 'Texto'])
    TabelaProposicoes = base.append(TabelaProposicoes).fillna('')
    
    conn = pyodbc.connect('Trusted_Connection=yes',
                      driver = '{ODBC Driver 17 for SQL Server}',
                      server = 'localhost',
                      database = 'Indaial')
    
    cursor = conn.cursor()
    
    for index, row in TabelaProposicoes.iterrows():
                
        cursor.execute('''
            
            INSERT INTO Proposicoes(
                DataReuniao,
                DataDeliberacao,
                Situacao,
                Assunto,
                Autor,
                Proposicao,
                Ano,
                Texto
            )
            values(?,?,?,?,?,?,?,?)''',
                       
            row['Reunião'],
            row['Deliberação'],
            row['Situação'],
            row['Assunto'],
            row['Autor'],
            row['Proposição'],
            row['Ano'],
            row['Texto']
                       
        )
        
    conn.commit()
    cursor.close()

In [167]:
# chamar a função select
SQLSelect('''
    select 
        *
    from Proposicoes
''')

SyntaxError: invalid syntax (<ipython-input-167-da307f55e209>, line 5)

In [14]:
# Função para truncar tabela no SQL
def SQLTruncate(NomeTabela):
    
    conn = pyodbc.connect('Trusted_Connection=yes', 
                          driver = '{ODBC Driver 17 for SQL Server}',
                          server = 'localhost', 
                          database = 'Indaial')
    
    cursor = conn.cursor()
    
    cursor.execute(f'''
                    
                    TRUNCATE TABLE {NomeTabela}
                    
                    ''')
    
    conn.commit()
    cursor.close()
    

In [28]:
#chamar a função truncate
# SQLTruncate('Proposicoesl')

In [12]:
# função que utiliza os métodos criados para inserir os dados de um ano inteiro no banco de dados atreavés de um loop  
def BuscaGravaDados(ano, quantidade = 10, erros_admissiveis = 2, segundos_espera = 0.5):
    
    erros = 0
    next_prop = 0
    
    lista = []
    i = 1
    
    while erros <= erros_admissiveis:
        
        try:
            dados_ano = SQLSelect(f'select Proposicao = max(Proposicao) from Proposicoes where Ano = {ano}')
            ultima_proposicao = dados_ano['Proposicao'].loc[0]
            
            if ultima_proposicao == None:                           # estrutura 1 p/ começar a inserção pelo 1° dado
                proxima_proposicao = 1  
            elif next_prop == 1:                                    # estrutura 2 p/ evitar que dados vazios sejam inseridos no banco de dados 
                proxima_proposicao = ultima_proposicao + 2
                next_prop = 0
            else:
                proxima_proposicao = ultima_proposicao + 1          # estrutura 3 p/ começar a inserção pelo último dado encontrado no banco de dados
                
            dados = Conteudo(proxima_proposicao, ano)
            if len(dados) == 0:
                erros += 1                      # estrutura para retornar o erro p/ as variáveis que são parâmetros na estrutura anterior
                next_prop = 1
            else:
                tabela = pd.DataFrame([dados])
                SQLInsertProposicoes(tabela)
        except:
            erros += 1
            pass
        
        time.sleep(segundos_espera)
        i += 1

In [17]:
# loop para automatizar a inserção dos dados utilizando uma lista de anos 
ano_inicial = 1996
ano_final = 1996

for i in list(range(ano_inicial, ano_final+1)):
    print('Iniciando gravação dos dados do ano: ',i)
    try:
        BuscaGravaDados(i, quantidade = 999999)
    except:
        pass
    
print('Inserção finalizada 😁')

Iniciando gravação dos dados do ano:  1996
Inserção finalizada 😁
